In [72]:
from numba import njit

@njit
def scale():
     [ndi.gaussian_filter(img, s) for s in range(5,50,5)]

# @njit
# def scale2():
    

In [115]:
def shift5(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num, :,:] = fill_value
        result[num:,:,:] = arr[:-num, :,:]
    elif num < 0:
        result[num:, :,:] = fill_value
        result[:num, :,:] = arr[-num:, :,:]
    else:
        result[:] = arr
    return result

@njit
def dog(gaussian_images, sigma_list):
    for i in range(len(sigma_list)-2):
        gaussian_images[:, :, i] = \
            (gaussian_images[:,:, i] - gaussian_images[:,:, i+1]) * sigma_list[i] / 0.109
    return gaussian_images[:,:,:-2]

In [126]:
import time
import numpy as np
from scipy import ndimage as ndi
import cv2
import multiprocessing as mp
from functools import partial
import itertools as it

# img = np.asarray([np.random.normal(0, 1, (256,256)) for _ in range(10)])
img = np.random.normal(0, 1, (256,256))
sigma_list = [i for i in range(2,12,1)]

start = time.time()
mean = []
for i in range(30):
    ks = [int(4 * s + 0.5) for s in sigma_list]
    ks = [kl if kl % 2 == 1 else kl + 1 for kl in ks]
    # partial_f = partial(cv2.GaussianBlur, src=img)
    with mp.Pool(8) as pool:
        gaussian_images = pool.starmap(cv2.GaussianBlur, zip(it.repeat(img) ks, sigma_list))
    for m, s in enumerate(sigma_list[:-1]):
        gaussian_images[m] = (gaussian_images[m] - gaussian_images[m+1]) * sigma_list[m] / 0.109
    dog_images = np.asarray(gaussian_images[:-1])
    mean.append(time.time()- start)
print(f'mean: {np.mean(mean)}, std: {np.std(mean)}')


start = time.time()
mean = []
for i in range(30):
    gaussian_images = []
    for m, s in enumerate(sigma_list):
        ks = int(4 * s + 0.5) 
        ks = ks if ks % 2 == 1 else ks + 1
        gaussian_images.append(cv2.GaussianBlur(img, (ks, ks), s))
        if m != 0:
            gaussian_images[m - 1] = (gaussian_images[m-1] - gaussian_images[m]) * sigma_list[m-1] / 0.109
    dog_images = np.asarray(gaussian_images[:-1])
    mean.append(time.time()- start)
print(f'mean: {np.mean(mean)}, std: {np.std(mean)}')


# img_temp = img[]
# dog_images = [(img[i] - img[i + 1]) * np.mean(sigma_list[i])/0.5 for i in range(img.shape[2]-1)]
# dog_images = np.asarray(dog_images)


start = time.time()
mean = []
for i in range(30):
    gaussian_images = np.empty((img.shape[0], img.shape[1], len(sigma_list)))
    dog_images = np.empty((img.shape[0], img.shape[1], len(sigma_list)-1))
    for s in sigma_list:
        ks = int(4 * s + 0.5) 
        ks = ks if ks % 2 == 1 else ks + 1
        gaussian_images[:, :, m - 1] = cv2.GaussianBlur(img, (ks, ks), s)
    dog_images = dog(gaussian_images, np.array(sigma_list))
    mean.append(time.time()- start)
print(f'mean: {np.mean(mean)}, std: {np.std(mean)}')

# start = time.time()
# mean = []
# for i in range(1000):
#     np.array([0.1 * (0.5 ** i) for i in range(10)])
#     mean.append(time.time()- start)
# print(f'mean: {np.mean(mean)}, std: {np.std(mean)}')

SyntaxError: invalid syntax (4025832330.py, line 20)

In [84]:
img.shape[2]

256